In [556]:
import os
from os.path import join
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import matplotlib.pyplot as plt

from PIL import Image


In [557]:
class directionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4*4*4,64)# output Layer to Intermediate modules
        self.fc2 = nn.Linear(64,64) #Intermediate modules to Output Layer
        self.fc3 = nn.Linear(64,32) #Intermediate modules to Output Layer
        self.fc4 = nn.Linear(32,32) #Intermediate modules to Output Layer
        self.fc5 = nn.Linear(32,16) #Intermediate modules to Output Layer
        self.fc6 = nn.Linear(16,16)
        self.fc7 = nn.Linear(16,8)
        self.fc8 = nn.Linear(8,8)
        self.fc9 = nn.Linear(8,5)  

    def forward(self, x):#順伝播 Forward propagation
        x = x.view(-1,4*4*4)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc7(x)
        x = self.fc8(x)
        y = self.fc9(x)
        return y

In [558]:
class sellectionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4*4*3,48)# Input Layer to Intermediate modules
        self.fc2 = nn.Linear(48,48) #Intermediate modules to Output Layer
        self.fc3 = nn.Linear(48,24) #Intermediate modules to Output Layer
        self.fc4 = nn.Linear(24,24) #Intermediate modules to Output Layer
        self.fc5 = nn.Linear(24,16) #Intermediate modules to Output Layer
        self.fc6 = nn.Linear(16,16)

    def forward(self, x):#順伝播 Forward propagation
        x = x.view(-1,4*4*3)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        y = self.fc6(x)
        #y = nn.Softmax(self.fcp(x))
        return y

In [559]:
sellectNet = sellectionModel()
directionNet = directionModel()


In [560]:
sm_state_dict = torch.load('4x4_sellect_v1_model.pth')
sellectNet.load_state_dict(sm_state_dict)
dm_state_dict = torch.load('4x4_direction_v1_model.pth')
directionNet.load_state_dict(dm_state_dict)

<All keys matched successfully>

In [561]:
def get_img(path):
    img = Image.open(path)
    img = np.array(img)
    return img
def get_tensor(path):
    tensor = torch.load(path)
    tensor = np.array(tensor)
    tensor = tensor.astype(np.float32)
    return tensor

In [562]:
path = "E:/procon/data/4x4_v1/eve/1.pt"
problem = get_tensor(path)
print(type(problem[0]))
print(problem)
def print_2d(my_list):
    for a in my_list:
        print(*a) 


<class 'numpy.ndarray'>
[[[ 3.  0. 12.]
  [ 0.  3.  3.]
  [ 2.  3. 11.]
  [ 1.  2.  6.]]

 [[ 2.  1.  9.]
  [ 0.  1.  1.]
  [ 0.  0.  0.]
  [ 1.  3.  7.]]

 [[ 2.  2. 10.]
  [ 1.  1.  5.]
  [ 0.  2.  2.]
  [ 2.  0.  8.]]

 [[ 1.  0.  4.]
  [ 3.  2. 14.]
  [ 3.  1. 13.]
  [ 3.  3. 15.]]]


In [563]:
#ゼロをすべてのマスの最後にたす(関数＋処理)
def addzero(n):
    n = list(np.append(n,0))
    return n

def addZero4all(n):
    n = list(map(addzero,n))
    return n

def addzero_power(n):
    h = len(n)
    addzerod  = []
    for i in range(h):
        addzerod.append(list(map(addzero,n[i])))
    
    return addzerod

#すべてのマスの末尾を消す　上の反対をする
def del_last(n):
    del n[-1]
    return n

def del_all_last(n):
    for i in range(len(n)):
        n[i] = list(map(del_last,n[i]))
    return n

In [564]:
#sellectNetをつかい選択されたますの末尾を255にする(関数)
def sellect(input_array):
    call_array = np.array(del_all_last(copy.deepcopy(input_array))).astype(np.float32)
    sellected = sellectNet(torch.Tensor(call_array))
    target_index = sellected.argmax()
    w = len(input_array[0])
    target = [target_index//w,target_index%w]
    input_array[target[0]][target[1]][3] = 255

    return input_array, target


In [565]:
#directionNetの結果に基づいて交換する(関数)
def move(input_array, target):
    w = len(input_array)
    moveList = ["D","L","U","R",False]
    # if input_array[target[0]][target[1]][0] < target[0] and input_array[target[0]-1][target[1]][0] <= target[0]:
    #     way = 2
    # elif input_array[target[0]][target[1]][0] > target[0] and input_array[lambda x :target[0]+1 if target[0]+1 < len(input_array) else 0][target[1]][0] >= target[0]:
    #     way = 0
    # if input_array[target[0]][target[1]][1] < target[1] and input_array[target[0]][target[1]][0-1] <= target[1]:
    #     way = 1
    # elif input_array[target[0]][target[1]][1] > target[1] and input_array[target[0]][lambda x :target[1]+1 if target[1]+1 < len(input_array[0]) else 0][0] >= target[1]:
    #     way = 3
    
    next_is_collect = False
    if lambda a :target[0]+1 if target[0]+1 < len(input_array) else 0 == target[0]:
        if lambda b :target[1]+1 if target[1]+1 < len(input_array[0]) else 0 == target[1]:
            next_is_collect = True
            
    if input_array[target[0]][target[1]][0] < target[0] :
        way = 2
    elif input_array[target[0]][target[1]][0] > target[0] :
        way = 0
    if input_array[target[0]][target[1]][1] < target[1] :
        way = 1
    elif input_array[target[0]][target[1]][1] > target[1] :
        way = 3
    else:
        way =  4
    #if input_array[target[0]][target[1]][0] == target[0] and input_array[target[0]][target[1]][1] == target[1]:
    
    change = moveList[way]

    if way == 0:
        if target[0]+1 == w:
            input_array[target[0]][target[1]], input_array[0][target[1]] = input_array[0][target[1]], input_array[target[0]][target[1]]
            #output_array[target[0]][target[1]], output_array[0][target[1]] = output_array[0][target[1]], output_array[target[0]][target[1]]
            target = [0,target[1]]
        else:
            input_array[target[0]][target[1]], input_array[target[0]+1][target[1]] = input_array[target[0]+1][target[1]], input_array[target[0]][target[1]]
            #output_array[target[0]][target[1]], output_array[target[0]][target[1]+1] = output_array[target[0]][target[1]], output_array[target[0]][target[1]+1]
            target = [target[0]+1,target[1]]
    elif  way == 1:
        #print(input_array[target[0]][target[1]])
        input_array[target[0]][target[1]], input_array[target[0]][target[1]-1] = input_array[target[0]][target[1]-1], input_array[target[0]][target[1]]
        #output_array[target[0]][target[1]], output_array[target[0]][target[1]-1] = output_array[target[0]][target[1]], output_array[target[0]][target[1]-1]
        target = [target[0],target[1]-1]
    elif way == 2:
        input_array[target[0]][target[1]], input_array[target[0]-1][target[1]] = input_array[target[0]-1][target[1]], input_array[target[0]][target[1]]
        #output_array[target[0]][target[1]], output_array[target[0]-1][target[1]] = output_array[target[0]-1][target[1]], output_array[target[0]][target[1]]
        target = [target[0]-1,target[1]]
    elif way == 3:
        if target[1]+1 == len(input_array[0]):
            input_array[target[0]][target[1]], input_array[target[0]][0] = input_array[target[0]][0], input_array[target[0]][target[1]+1]
            #output_array[target[0]][target[1]], output_array[target[0]][0] = output_array[target[0]][0], output_array[target[0]][target[1]+1]
            target = [target[0],0]
        else:
            input_array[target[0]][target[1]], input_array[target[0]][target[1]+1] = input_array[target[0]][target[1]+1], input_array[target[0]][target[1]]
            #output_array[target[0]-1][target[1]], output_array[target[0]][target[1]] = output_array[target[0]-1][target[1]], output_array[target[0]][target[1]]
            target = [target[0],target[1]+1]
    elif way == 4:
        input_array[target[0]][target[1]][3] = 0
    

    
    return input_array, change, target

In [566]:
#directionNetの結果に基づいて交換する(関数)
def moveNet(input_array, target):
    call_array = np.array(copy.deepcopy(input_array)).astype(np.float32)
    direction = directionNet(torch.tensor(call_array))
    way = direction.argmax()
    w = len(input_array)
    moveList = ["D","L","U","R",False]
    change = moveList[way]
    if way == 0:
        if target[0]+1 == w:
            input_array[target[0]][target[1]], input_array[0][target[1]] = input_array[0][target[1]], input_array[target[0]][target[1]]
            #output_array[target[0]][target[1]], output_array[0][target[1]] = output_array[0][target[1]], output_array[target[0]][target[1]]
            target = [0,target[1]]
        else:
            input_array[target[0]][target[1]], input_array[target[0]+1][target[1]] = input_array[target[0]+1][target[1]], input_array[target[0]][target[1]]
            #output_array[target[0]][target[1]], output_array[target[0]][target[1]+1] = output_array[target[0]][target[1]], output_array[target[0]][target[1]+1]
            target = [target[0]+1,target[1]]
    elif  way == 1:
        #print(input_array[target[0]][target[1]])
        input_array[target[0]][target[1]], input_array[target[0]][target[1]-1] = input_array[target[0]][target[1]-1], input_array[target[0]][target[1]]
        #output_array[target[0]][target[1]], output_array[target[0]][target[1]-1] = output_array[target[0]][target[1]], output_array[target[0]][target[1]-1]
        target = [target[0],target[1]-1]
    elif way == 2:
        input_array[target[0]][target[1]], input_array[target[0]-1][target[1]] = input_array[target[0]-1][target[1]], input_array[target[0]][target[1]]
        #output_array[target[0]][target[1]], output_array[target[0]-1][target[1]] = output_array[target[0]-1][target[1]], output_array[target[0]][target[1]]
        target = [target[0]-1,target[1]]
    elif way == 3:
        if target[1]+1 == len(input_array[0]):
            input_array[target[0]][target[1]], input_array[target[0]][0] = input_array[target[0]][0], input_array[target[0]][target[1]+1]
            #output_array[target[0]][target[1]], output_array[target[0]][0] = output_array[target[0]][0], output_array[target[0]][target[1]+1]
            target = [target[0],0]
        else:
            input_array[target[0]][target[1]], input_array[target[0]][target[1]+1] = input_array[target[0]][target[1]+1], input_array[target[0]][target[1]]
            #output_array[target[0]-1][target[1]], output_array[target[0]][target[1]] = output_array[target[0]-1][target[1]], output_array[target[0]][target[1]]
            target = [target[0],target[1]+1]
    elif way == 4:
        input_array[target[0]][target[1]][3] = 0

    
    return input_array, change, target

In [567]:
#Main
path = "E:/procon/data/4x4_v1/eve/1.pt"
problem = get_tensor(path)
problem  = addzero_power(problem)
i = 0
k = 0
while True:
    #print(i)
    #print(type(problem))
    problem, target = sellect(problem)
    problem, direction, target = move(problem, target)
    k += 1
    while direction:
        #print("")
        problem, direction, target = move(problem, target)
        i += 1
        #print_2d(problem)
    if i > 8 or k >30:
        break

for i in range(4):
    for j in range(4):
        print(problem[i][j][2])

12.0
3.0
11.0
6.0
0.0
9.0
1.0
7.0
8.0
5.0
2.0
10.0
4.0
14.0
13.0
15.0


In [568]:
print(problem)

[[[3.0, 0.0, 12.0, 0.0], [0.0, 3.0, 3.0, 0.0], [2.0, 3.0, 11.0, 0.0], [1.0, 2.0, 6.0, 0.0]], [[0.0, 0.0, 0.0, 0], [2.0, 1.0, 9.0, 0.0], [0.0, 1.0, 1.0, 0.0], [1.0, 3.0, 7.0, 0.0]], [[2.0, 0.0, 8.0, 0], [1.0, 1.0, 5.0, 0], [0.0, 2.0, 2.0, 0], [2.0, 2.0, 10.0, 0.0]], [[1.0, 0.0, 4.0, 0.0], [3.0, 2.0, 14.0, 0.0], [3.0, 1.0, 13.0, 0.0], [3.0, 3.0, 15.0, 0.0]]]
